In [90]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

import tkinter as tk
from tkinter import filedialog as fd

import collections

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

In [91]:
# open file dialog / catastro de avance
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".shp", ".shp")])
root.destroy()

# open file dialog / xlsx de avance
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()

In [5]:
file_cat_avance = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/SHP/cosecha090623.shp'
file_xlsx_avance = r'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO 2023/REPORTE_AVANCE_COSECHA/RENDIMIENTO AVANCE 090623.xlsx'

In [6]:
catastro_avance = gpd.read_file(file_cat_avance)

In [8]:
data_avance = pd.read_excel(file_xlsx_avance)

In [9]:
# extraer codigos de cosecha
codigos_cosecha = list(set(data_avance['COD COS']))

In [10]:
print(codigos_cosecha)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226,

## COMPROMISO Y ENTREGAS

In [11]:
# seleccion de grupo aguilera
cod_co = 30
# filtrado por codigo de cosecha
grupo = data_avance[data_avance['COD COS'] == cod_co]
# seleccion de columnas de interes
grupo = grupo[['CODIGO CAÑERO ', 'COD COS', 'NOMBRE CAÑERO', 'TN COMPROMISO INDIVIDUAL', 'TN ENTREGA INDIVIDUAL']]
# calculo de % de avance de entrega del grupo
grupo['avance_porcen'] = (grupo['TN ENTREGA INDIVIDUAL'] / grupo['TN COMPROMISO INDIVIDUAL'])*100
# cambio del nombre del campo codigo cañero
grupo = grupo.rename(columns={'CODIGO CAÑERO ': 'cod_ca'})
grupo

,cod_ca,COD COS,NOMBRE CAÑERO,TN COMPROMISO INDIVIDUAL,TN ENTREGA INDIVIDUAL,avance_porcen
0,75,30,AGUILERA TARADELLES NELSON MARIANO,26992.97,0.00,0.000000
1,577,30,AGUILERA MARIA INGRID WENDE DE,13433.25,631.64,4.702064
2,794,30,AGUILERA WENDE MARIANO,18084.70,15218.32,84.150249
3,8005,30,AGUILERA WENDE MARIA INGRID,5201.02,20.45,0.393192
4,41715,30,VACA DIEZ UGARTE NICOLAS,1781.38,0.00,0.000000


In [12]:
# totales de tabla de entreas del grupo seleccionado
grupo_totales = {
    'total_compromiso':grupo['TN COMPROMISO INDIVIDUAL'].sum(),
    'total_entrega':grupo['TN ENTREGA INDIVIDUAL'].sum(),
    'total_avance_porcen':grupo['TN ENTREGA INDIVIDUAL'].sum() / grupo['TN COMPROMISO INDIVIDUAL'].sum() * 100
}
grupo_totales

{'total_compromiso': 65493.32,
 'total_entrega': 15870.41,
 'total_avance_porcen': 24.232104892529495}

## AVANCE PROPIEDAD

In [13]:
hoy = datetime.now()
mes_actual = hoy.month
mes_actual

7

In [14]:
# cambio de nombre de columna para hacer merge con la tabla de grupo
catastro_avance = catastro_avance.rename(columns={'unidad_03':'cod_ca'})
# merge 
catastro_merge = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge['area_cosechada'] = catastro_merge.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# calculo de area cosechada en el mes actual
catastro_merge['avance_mes'] = catastro_merge.apply(lambda row : row['area'] if row['fc']!=None and row['fc'].split('-')[1] == str(mes_actual) else 0, axis=1)
# agrupacion de datos por pripiedad, resumiendo por area, area_cosechada, avance_mes
propiedades = catastro_merge.groupby(['unidad_01', 'unidad_02'])[['area', 'area_cosechada', 'avance_mes']].sum()
# reseteo de index
propiedades = propiedades.reset_index()
# calculo de % de avance de cosecha
propiedades['%avance'] = propiedades['area_cosechada'] / propiedades['area'] * 100
propiedades

,unidad_01,unidad_02,area,area_cosechada,avance_mes,%avance
0,216,LA_ESTACADA--AGUILERA,226.34,0.00,0,0.000000
1,217,FLAMBOYAN--AGUILERA,207.46,0.00,0,0.000000
2,256,LOS_ANDES--AGUILERA,423.24,153.93,0,36.369436


In [16]:
# totales de tabla por columna
propiedades_totales = {
    'total_area':propiedades['area'].sum(),
    'total_area_cosechada':propiedades['area_cosechada'].sum(),
    'total_avance_mes':propiedades['avance_mes'].sum(),
    'total_avance_porcen':propiedades['area_cosechada'].sum() / propiedades['area'].sum() * 100
}
propiedades_totales

{'total_area': 857.04,
 'total_area_cosechada': 153.93,
 'total_avance_mes': 0,
 'total_avance_porcen': 17.96065527863344}

# AVANCE POR VARIEDAD

In [43]:
# merge 
catastro_merge_2 = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge_2['area_cosechada'] = catastro_merge_2.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)
# calculo de area cosechada en el mes actual
#catastro_merge_2['avance_mes'] = catastro_merge_2.apply(lambda row : row['area'] if row['fc']!=None and row['fc'].split('-')[1] == str(mes_actual) else 0, axis=1)
# agrupacion de datos por pripiedad, resumiendo por area, area_cosechada, avance_mes
avance_variedad = catastro_merge.groupby(['variedad'])[['area_cosechada', 'area']].sum()
# reseteo de index
avance_variedad = avance_variedad.reset_index()
# ordemar por area cosechadad
avance_variedad = avance_variedad.sort_values('area_cosechada', ascending=False)
# calculo de porcentaje de avance
avance_variedad['porcen_avan'] = avance_variedad.apply(lambda row : row['area_cosechada']/row['area'] *100, axis=1)
avance_variedad

,variedad,area_cosechada,area,porcen_avan
6,UCG_90-20,93.26,467.48,19.949517
1,CITTCA_85-22,46.45,140.07,33.161990
5,SP_83-5073,14.22,81.55,17.437155
0,CITTCA-0563,0.00,15.51,0.000000
2,RB-2,0.00,135.15,0.000000
3,RB-4,0.00,4.25,0.000000
4,RBB_77-26,0.00,13.03,0.000000


In [52]:
totales_variedad = {
    'total_cosechado' : avance_variedad['area_cosechada'].sum(),
    'total_area' : avance_variedad['area'].sum(),
    'total_porcen' : avance_variedad['area_cosechada'].sum() / avance_variedad['area'].sum() * 100
}
totales_variedad

{'total_cosechado': 153.93,
 'total_area': 857.0399999999998,
 'total_porcen': 17.96065527863344}

## AVANCE POR CORTE

In [88]:
# merge 
catastro_merge_3 = pd.merge(grupo, catastro_avance, on='cod_ca')
# crea nueva columna "area_cosecha", se almacena en area si el campo estado="cosechado"
catastro_merge_3['area_cosechada'] = catastro_merge_3.apply(lambda row : row['area'] if row['estado'] == 'cosechado' else 0, axis=1)

suma_area = catastro_merge_3.loc[catastro_merge_3['soca'] >= 10, 'area'].sum()
suma_area_cosechada = catastro_merge_3.loc[catastro_merge_3['soca'] >= 10, 'area_cosechada'].sum()

catastro_merge_3= catastro_merge_3[catastro_merge_3['soca'] < 10]

catastro_merge_3['soca'] = catastro_merge_3['soca'].astype(str)

new_reg = {'soca': '>10', 'area': suma_area, 'area_cosechada': suma_area_cosechada}

catastro_merge_3 = catastro_merge_3.append(new_reg, ignore_index=True)
#agrupacion de datos por pripiedad, resumiendo por area, area_cosechada, avance_mes
avance_corte = catastro_merge_3.groupby(['soca'])[['area_cosechada', 'area']].sum()
# reseteo de index
avance_corte = avance_corte.reset_index()
# ordemar por area cosechadad
avance_corte = avance_corte.sort_values('soca')
# calculo de porcentaje de avance
avance_corte['porcen_avan'] = avance_corte.apply(lambda row : row['area_cosechada']/row['area'] *100, axis=1)
#avance_corte

In [89]:
avance_corte

,soca,area_cosechada,area,porcen_avan
0,0,62.30,144.88,43.001104
1,1,46.41,167.70,27.674419
2,2,0.00,103.16,0.000000
3,3,0.00,81.21,0.000000
4,4,0.00,107.61,0.000000
5,5,16.98,50.66,33.517568
6,6,0.00,46.77,0.000000
7,7,6.17,33.71,18.303174
8,8,0.00,14.81,0.000000
9,9,0.00,65.66,0.000000


In [90]:
filename = str(cod_ca) + '_ADC_' + '09-06-2023_' + nom_ca
filename

'75_ADC_09-06-2023_AGUILERA TARADELLES NELSON MARIANO'

In [91]:
df

,unidad_01,unidad_02,area,area_cosechada,TCH,sin_cosechar,produccion_total,produccion_restante,produccion_cosechado
0,216,LA_ESTACADA--AGUILERA,226.34,0.00,67.544873,226.34,15288.106548,15288.106548,0.00000
1,217,FLAMBOYAN--AGUILERA,207.46,0.00,67.975285,207.46,14102.152708,14102.152708,0.00000
2,256,LOS_ANDES--AGUILERA,423.24,153.93,79.795302,269.31,33772.563435,21489.672665,12282.89077


In [92]:
totales = {
    'tn_compromiso': lista_ca['tn_compromiso_indiv'].sum(),
    'tn_entregado': lista_ca['tn_entregado_indiv'].sum(),
    'avance': (lista_ca['tn_entregado_indiv'].sum()  / lista_ca['tn_compromiso_indiv'].sum())*100,
    'area': df['area'].sum(),
    'area_cosechada': df['area_cosechada'].sum(),
    'sin_cosechar': df['sin_cosechar'].sum(),
    'produccion_total': df['produccion_total'].sum(),
    'produccion_restante': df['produccion_restante'].sum(),
    'produccion_cosechado': df['produccion_cosechado'].sum(),
    'fecha': '09/06/2023'
}
total = collections.namedtuple("totales", totales.keys())(*totales.values())

In [93]:
canheros = []
for index, row in lista_ca.iterrows():
    ca = dict(row)
    canheros.append(collections.namedtuple("canhero", ca.keys())(*ca.values()))

In [94]:
canheros

[canhero(cod_ca=75, cod_cos=30, nombre_canhero='AGUILERA TARADELLES NELSON MARIANO', tn_compromiso_indiv=26992.97, tn_entregado_indiv=0.0, avance_porcen=0.0),
 canhero(cod_ca=577, cod_cos=30, nombre_canhero='AGUILERA MARIA INGRID WENDE DE', tn_compromiso_indiv=13433.25, tn_entregado_indiv=631.64, avance_porcen=4.702063908585041),
 canhero(cod_ca=794, cod_cos=30, nombre_canhero='AGUILERA WENDE MARIANO', tn_compromiso_indiv=18084.7, tn_entregado_indiv=15218.32, avance_porcen=84.15024855264394),
 canhero(cod_ca=8005, cod_cos=30, nombre_canhero='AGUILERA WENDE MARIA INGRID', tn_compromiso_indiv=5201.02, tn_entregado_indiv=20.45, avance_porcen=0.3931921046256311),
 canhero(cod_ca=41715, cod_cos=30, nombre_canhero='VACA DIEZ UGARTE NICOLAS', tn_compromiso_indiv=1781.38, tn_entregado_indiv=0.0, avance_porcen=0.0)]

In [95]:
propiedades = []
for index, row in df.iterrows():
    prop = dict(row)
    propiedades.append(collections.namedtuple("propiedad", prop.keys())(*prop.values()))

In [96]:
propiedades

[propiedad(unidad_01=216, unidad_02='LA_ESTACADA--AGUILERA', area=226.34, area_cosechada=0.0, TCH=67.54487297159044, sin_cosechar=226.34, produccion_total=15288.10654838978, produccion_restante=15288.10654838978, produccion_cosechado=0.0),
 propiedad(unidad_01=217, unidad_02='FLAMBOYAN--AGUILERA', area=207.45999999999998, area_cosechada=0.0, TCH=67.97528539650631, sin_cosechar=207.45999999999998, produccion_total=14102.152708359197, produccion_restante=14102.152708359197, produccion_cosechado=0.0),
 propiedad(unidad_01=256, unidad_02='LOS_ANDES--AGUILERA', area=423.24, area_cosechada=153.93, TCH=79.79530156723294, sin_cosechar=269.31, produccion_total=33772.56343531567, produccion_restante=21489.672665071503, produccion_cosechado=12282.890770244167)]

In [97]:
doc = DocxTemplate("tpl_reporte.docx")

In [98]:
context = {'canheros' : canheros, 'propiedades' : propiedades, 'total':total}
doc.render(context)
doc.save('reportes/' + filename + '.docx')

# Convertir a PDF

In [249]:
import os

In [250]:
from docx2pdf import convert

In [251]:
path = r'reportes/'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

SE ENCONTRARON 1 INFORMES
1 INFORMES


['reportes//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.docx']

In [252]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdfs'))
    pdf = filename.replace('.docx','.pdf').split('/')[-1]
    convert(filename, r'pdf/' + pdf)
    i+=1
print('END')

CONVIRTIENDO... INFORME 1 reportes//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.docx
reportes//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.docx
reportes//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf


  0%|          | 0/1 [00:00<?, ?it/s]

END


# leer ruta de planos

In [253]:
path = r'pdf/'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')

SE ENCONTRARON 1 INFORMES
1 INFORMES


In [254]:
reporte = rutas[0]

In [255]:
reporte

'pdf//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf'

In [256]:
list_pdfs = []
list_pdfs.append(reporte)

In [257]:
list_pdfs

['pdf//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf']

In [258]:
path = r'D:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2023\ANALISIS DE CAÑA EN CAMPO 2023\REPORTE_AVANCE_COSECHA\PLANOS'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')

SE ENCONTRARON 1578 INFORMES
1578 INFORMES


In [259]:
data  = {}
for i in rutas:
    cod_prop = i.split('_')[-1].split('.')[0]
    data[cod_prop]=i

In [260]:
data['30']

'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_30.pdf'

In [261]:
df

,unidad_01,unidad_02,area,area_cosechada,TCH,sin_cosechar,produccion_total,produccion_restante,produccion_cosechado
0,732,EL_PUESTO--BALCAZAR,18.59,1.22,46.944692,17.37,872.701825,815.429300,57.272524
1,733,LAS_PARCELAS--BALCAZAR,20.18,0.00,45.217658,20.18,912.492343,912.492343,0.000000
2,749,SAN_JORGE_DE_NAICO(EL_TRILLO)--BALCAZAR,25.09,4.28,43.599069,20.81,1093.900635,907.296621,186.604014
3,750,EL_CARMEN_NAICO--BALCAZAR,7.06,0.00,47.181886,7.06,333.104118,333.104118,0.000000
4,1694,EL_TRILLO--BALCAZAR_RAMON,25.88,1.24,48.479291,24.64,1254.644064,1194.529742,60.114321
5,1818,EL_PUESTO--BALCAZAR_RAMON,8.10,4.04,45.167463,4.06,365.856452,183.379901,182.476551
6,1819,EL_PUESTO--BALCAZAR_SOLETO_MARIO,3.54,0.00,39.509156,3.54,139.862413,139.862413,0.000000
7,1820,SAN_JORGE--BALCAZAR_SOLETO_MARIO,4.71,0.00,53.842054,4.71,253.596073,253.596073,0.000000


In [262]:
for row in df.itertuples():
    cod_prop = row.unidad_01
    ruta_prop = data[str(cod_prop)]
    list_pdfs.append(ruta_prop)

In [263]:
list_pdfs

['pdf//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_732.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_733.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_749.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_750.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_1694.pdf',
 'D:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - AVANCE COSECHA\\2023\\ANALISIS DE CAÑA EN CAMPO 2023\\REPORTE_AVANCE_COSECHA\\PLANOS/PLANO_1818.pdf',
 'D:\\Inge

In [264]:
from PyPDF2 import PdfMerger

In [265]:
filename.replace('reportes', 'pdf_final').replace('docx', 'pdf')

'pdf_final//911_ADC_09-06-2023_BALCAZAR JUSTINIANO MARIO.pdf'

In [266]:
nombre_archivo_salida = filename.replace('reportes', 'pdf_final').replace('docx', 'pdf')

In [267]:
fusionador = PdfMerger()

In [268]:
for pdf in list_pdfs:
    fusionador.append(open(pdf, 'rb'))

In [269]:
with open(nombre_archivo_salida, 'wb') as salida:
    fusionador.write(salida)